# *Library and file dependancies*

In [ ]:
# !pip install kneed


*Libraries*

In [ ]:
# Single cell 
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
from kneed import KneeLocator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import ParameterGrid
# Consider aesthetics
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['axes.axisbelow'] = True
plt.rcParams.update({'font.size': 17})
# Output control
import warnings
warnings.filterwarnings("ignore")

*Single cell utility functions*

In [ ]:
from utils.plotting import *
from utils.optimization import *
from utils.stats import *

*data file path*

In [ ]:
filepath = 'datasets/'

# 1. **Preprocessing**

## **Data formatting** 

The original counts matrix is in txt format, where the columns are cells and the rows are genes. For this reason we load the dataset into a pandas dataframe and:
1. Remove the gene_id and gene_type columns
2. Set the gene name column as the index
3. Transpose the Dataframe

Finally we can load the Dataframe into an AnnData object, readable by scanpy. 

In [ ]:
# Pass the txt file into a csv via pandas with tab delimeter
df = pd.read_csv(filepath+"GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt", delimiter="\t")
df.drop(['gene_id','gene_type'], axis = 1, inplace = True)
df.set_index('gene_name', drop=True, inplace=True)
df = df.rename_axis(None)
df  = df.T 
adata = sc.AnnData(df.values)
adata.var_names = df.columns  # Assign column names as feature (gene) names
adata.obs_names = df.index  # Assign row names as observation names (cell or sample names)
adata.var_names_make_unique() # In case of repeatring genes, make the unique
print(adata)
print("\n---------------------\n")
print(adata.obs_names)

We have a counts matrix with 563 observations and 57915 genes. As you can see in the adata object observation we also have pooled tumour samples. Thus we will seperate the information into two adata objects:
- One for pooled cell counts `padata` 
- One for cell counts `cadata`

In [ ]:
# Cell adata 
cadata = adata[14:]
print(f"Cell adata obvervations: {cadata.obs_names}\n")
padata = adata[:14]
print(f"Pool cell adata observations: {padata.obs_names}\n")

In [ ]:
df = pd.read_csv(filepath+"GSE75688_final_sample_information.txt", delimiter="\t")
df_sorted = df.sort_values(by='sample', ascending=True)
df_sorted = pd.concat([df[df['sample'].str.split('_', expand=True)[1] == 'Pooled'],
                      df[df['sample'].str.split('_', expand=True)[1] != 'Pooled']])

pooled_names = df.loc[df['sample'].str.contains('_Pooled'), 'sample']

mask = df['sample'].isin(pooled_names)

sorted_lines = df[mask].sort_values(by='sample')

df_sorted = pd.concat([sorted_lines, df[~mask]])
# rename the BC07_Pooled as BC07_Tumor
df_sorted.loc[129, 'sample'] = 'BC07_Tumor'

bulk_rows = df_sorted[df_sorted['type'] == "Bulk"]
rest_rows = df_sorted[df_sorted['type'] != "Bulk"]

df_sorted.head(25)

Another very important step is to assign each cell population to a tumor group. Cells are named by "BC":PREFIX + "02-11":PATIENT + "_" + "cell number", thus we can remove the specific cell number and only keep the patient tumor identifier "BC02" to "BC011".

In [ ]:
tumor_groups_c = [obs_name if "Pooled" in obs_name else obs_name.split("_")[0] for obs_name in cadata.obs_names]
cadata.obs['tumor_groups'] = tumor_groups_c
# For example the first gorup of cells should belong to the tumor of patient one 
cadata.obs['tumor_groups'].head()

## **Quality Control**

For quality control we utilize our own pipeline following single cell best practises, we end up with 518 cells rather than 515 of the Authors.

Before we begin we simply remove genes that have 0 expression across all cells since they can affect heavily the QC process

In [ ]:
mask = cadata.X.sum(axis = 0) == 0
cadata = cadata[:,~mask]
padata = padata[:,~mask]
cadata

## *Cell fileting*

As you can see this removed 21.792 genes that were not expressed in any of the cells

We continue with the QC:

To maintain quality control, we follow the best practices by removing cells from our analysis that show certain characteristics. These characteristics include having a small number of detected genes, low count depth, and a high proportion of mitochondrial counts. These traits may suggest that the cell membrane is compromised or that the cell is dead. Since these cells are not the main focus of our analysis and can affect the accuracy of our subsequent analysis, we remove them from our dataset using the three covariates.

  - The number of counts per cell (count depth)

  - The number of genes per cell

  - The fraction of counts from mitochondrial genes per cell <--- However in the context of breast cancer, since cancer cells may have altered mitchondrial pathways, we wont remove cells based on the last covariate

Below we annotate mitochondrial genes:

In [ ]:
# mitochondrial genes
cadata.var["mt"] = cadata.var_names.str.startswith("MT-")

Before proceeding with cell & gene filtering we annotate quality control metrics to the counts matrix via scanpy's `calculate_qc_metrics` for the `cadata` counts matrix for the top 20% genes.

In [ ]:
sc.pp.calculate_qc_metrics(cadata, qc_vars=["mt"], inplace=True, percent_top=[20], log1p=True
)
cadata

- "log1p total counts" refers to the log transformed overall count of all gene expression in a cell.
- The "log1p n genes by counts" value signifies the log transformed number of genes expressed in each cell.
- The "pct_counts_in_top_20_genes" value show the percentage of the total expression in each cell the 20 most expressed genes have.

Regarding the proportion of total counts in a cell that can be attributed to mitochondrial genes, we can see that we have outliers with very high mitochondrial gene expression

In [ ]:
sc.pl.scatter(cadata, y = 'log1p_n_genes_by_counts', x = 'log1p_total_counts', color='tumor_groups') 
# sc.pl.violin(cadata, ['n_genes_by_counts'], save='_n_genes', jitter=1)
# sc.pl.violin(cadata, ['total_counts'], save='_total_counts', jitter=1)



In [ ]:
sc.pl.violin(cadata, "log1p_total_counts", show =False, size = 5)
plt.title("Gene Expression Density")
plt.xlabel("Cells")
plt.ylabel("Density")
plt.show()

In [ ]:
sc.pl.violin(cadata, "log1p_n_genes_by_counts", show =False, size = 5)
plt.title("# Of Genes Density")
plt.xlabel("Cells")
plt.ylabel("Density")
plt.axhline(y=7.8, color='red', linestyle='--')

plt.show()

In [ ]:
sc.pl.violin(cadata, "pct_counts_in_top_20_genes", show =False, size = 5)
plt.title("% Total Expression of the 20 most expressed genes")
plt.xlabel("Cell")
plt.ylabel("Percentage")
plt.show()

In [ ]:
sc.pl.violin(cadata, "pct_counts_mt", show =False, size = 5)
plt.title("# Of Genes Density")
plt.xlabel("Cells")
plt.ylabel("Density")
plt.axhline(y=60, color='red', linestyle='--')

plt.show()

Based on the above, we remove cells that are located 5 absolute deviations from the median in any of the above distributions.
$$ MAD = median(|X_i - median(X))$$


In [ ]:
# cadata[(cadata.obs.log1p_n_genes_by_counts > 7.8) &
# (cadata.obs.pct_counts_mt < 60) ,:].shape
# (cadata.obs.pct_counts_mt < 65),:].shape

# cadata = cadata[(cadata.obs.log1p_total_counts_mt > 9.7) & 
# (cadata.obs.log1p_n_genes_by_counts > 7.7) &
# (cadata.obs.pct_counts_mt < 65),:]

In [ ]:
cadata.obs["outlier"] = (
    is_outlier(cadata, "log1p_total_counts", 5)
    | is_outlier(cadata, "log1p_n_genes_by_counts", 5)
    | is_outlier(cadata, "pct_counts_in_top_20_genes", 5)
)
cadata.obs.outlier.value_counts()

In [ ]:
cadata = cadata[~cadata.obs.outlier]

In [ ]:
# Convert "sample" column to a list
sample_values = rest_rows["sample"].tolist()

# Check if each value from cadata.obs_names is present in sample_values
is_present = cadata.obs_names.isin(sample_values)
count_true = is_present.sum()

# Print the result
print(count_true)

In [ ]:
# Apply the mask to the cadata
cadata = cadata[is_present]
cadata

## Same for Padata ##

In [ ]:
padata.var["mt"] = padata.var_names.str.startswith("MT-")

sc.pp.calculate_qc_metrics(padata, qc_vars=["mt"], inplace=True, percent_top=[20], log1p=True
)
padata

In [ ]:
padata.obs["outlier"] = (
     is_outlier(padata, "log1p_n_genes_by_counts", 5)
)
padata.obs.outlier.value_counts()

In [ ]:
sc.pl.violin(padata, "log1p_n_genes_by_counts", show =False, size = 5)
plt.axhline(y=9.5, color='red', linestyle='--')

In [ ]:
padata = padata[~padata.obs.outlier]

In [ ]:
# Convert "sample" column to a list
sample_values = bulk_rows["sample"].tolist()

# Check if each value from cadata.obs_names is present in sample_values
is_present = padata.obs_names.isin(sample_values)
count_true = is_present.sum()

# Print the result
print(count_true)

In [ ]:
# Apply the mask to the padata
padata = padata[is_present]
padata

## *Gene Filtering*

For gene filtering we follow the authors workflow and:
1. Set to 0 genes with counts < 1.
2. Remove genes that are expressed in less than 10% of all tumour groups. Below is an example of tumour expression for 10 of the total 36123 genes

In [ ]:
# 1. 
cadata.X[cadata.X < 1] = 0

In [ ]:
sc.pl.dotplot(cadata,cadata.var_names[:10],groupby="tumor_groups")

Now we can filter out all the genes expressed in 10% of all tumour groups.

In [ ]:
df_expression = pd.DataFrame(data=cadata.X, columns=cadata.var_names)
df_expression['tumor_groups'] = cadata.obs['tumor_groups'].values

# Group by tumor groups and sum the expression values for cells in the group
group = df_expression.groupby("tumor_groups").sum() 
# Make expression binary
group[group > 0] = 1
mask = group.sum(axis = 0) < 0.1 * group.shape[0] 
cadata = cadata[:,~mask]
padata = padata[:,~mask]
cadata

Finally we can perform log transformation and normalization

In [ ]:
cadata.X

In [ ]:
sc.pp.normalize_total(cadata)
sc.pp.log1p(cadata, base = 2)

sc.pp.normalize_total(padata)
sc.pp.log1p(padata, base = 2)

We obtained 27,000 genes out of the 57,000 total, which is higher than the 17,000 genes reported by the authors.

In [ ]:
padata

In [ ]:
sc.pp.pca(padata)
sc.pl.pca(padata,annotate_var_explained = True, title = "PCA, pooled counts",add_outline=True, size = 50,palette = "tab20")

In [ ]:
sc.pp.pca(cadata)
sc.pl.pca(cadata, color = "tumor_groups",annotate_var_explained = True, title = "PCA cell counts",add_outline=True, size = 50,palette = "tab20")

In [ ]:
sc.pl.pca_variance_ratio(cadata, log=True, n_pcs=50, show=False)
plt.axvline(x=8, color='red',linestyle='--')
plt.show()


Ανουσιο το κουμουλατιβ βαριανς πλοτ 

In [ ]:
# y = np.cumsum(cadata.uns['pca']['variance_ratio'])
# x = range(1,y.shape[0]+1)

# plt.figure(figsize=(15, 10), dpi=80)
# plt.plot(x, y, marker=',', linestyle='-', color='b')
# plt.ylim(0.0, 1.1)
# plt.xlabel('Number of Components')
# plt.xticks(np.arange(0, cadata.uns['pca']['variance_ratio'].shape[0]+1, step=10))
# plt.ylabel('Cumulative variance (%)')
# plt.title('PCA Variance vs. Number of Components')

# plt.axhline(y=0.95, color='red', linestyle='--')
# plt.text(0.5, 0.95, '95% cut-off threshold', color='red', fontsize=16)
# plt.axhline(y=0.80, color='green', linestyle='--')
# plt.text(0.5, 0.80, '80% cut-off threshold', color='green', fontsize=16)
# plt.axhline(y=1, color='black', linestyle='--')

# for x in np.arange(0, cadata.uns['pca']['variance_ratio'].shape[0]+1, step=10):
#     plt.axvline(x=x, color='gray', linestyle=':', alpha=0.5)

# plt.grid(axis='x')
# plt.tight_layout()
# plt.show()

## Save the processed adata objects ##

In [ ]:
adata.write("datasets/processed_cellcounts.h5ad")
adata.write("datasets/processed_pooledcounts.h5ad")

## UMAP ###

In [ ]:
umap_adata = cadata.copy()

# param_grid_nei = {'n_pcs': [9],'use_rep':[None],
#         'n_neighbors': [10,15,20,25,30,35,40]}
# param_grid_umap = {'min_dist':[0.5,0.75,1], 'random_state':[0]}

# for params in ParameterGrid(param_grid_nei):
#   for params_umap in ParameterGrid(param_grid_umap):
#     sc.pp.neighbors(umap_adata, **params)
#     sc.tl.umap(umap_adata,**params_umap)
#     title ='UMAP'+'_'+  str(params['n_neighbors'])+'_'+  str(params['n_pcs'])+'_'+ str(params_umap['min_dist'])
#     sc.pl.umap(umap_adata, color="tumor_groups", title=title)

sc.pp.neighbors(umap_adata, n_pcs=9, n_neighbors=10)
sc.tl.umap(umap_adata,min_dist=0.75, random_state=0 )

# # sc.pp.neighbors(umap_adata, n_pcs=9, n_neighbors=15)
# # sc.tl.umap(umap_adata,min_dist=1, random_state=0 )

# # sc.pp.neighbors(umap_adata, n_pcs=9, n_neighbors=20)
# # sc.tl.umap(umap_adata,min_dist=1, random_state=0 )


# # sc.pp.neighbors(umap_adata, n_pcs=9, n_neighbors=10)
# # sc.tl.umap(umap_adata,min_dist=0.5, random_state=0 )

sc.pl.umap(umap_adata, color="tumor_groups", title='UMAP 9n 6pc 0.75dist', add_outline=True, size = 50,palette = "tab20")


In [ ]:
sc.pp.neighbors(cadata, n_pcs=9, n_neighbors=10)
sc.tl.umap(cadata,min_dist=0.75, random_state=0 )

## t-SNE ##

In [ ]:
def tsne_params(datasize):
  n = datasize/100
  if n/100 > 30:
    perplexity = 30 + n/100 
  else:
    perplexity = 30
  if datasize/12 > 200:
    learning_rate = datasize/12
  else:
    learning_rate = 200
  return learning_rate, perplexity

In [ ]:
tsne_adata = umap_adata.copy()

lr, perpl = tsne_params(umap_adata.X.shape[0])


# n_pcs=[10,11,12,13,14,15,16,17,18,19,20]

# for par in n_pcs:
#     params = {'learning_rate': lr, 'perplexity': perpl, 'n_pcs': par}
#     sc.tl.tsne(tsne_adata, **params)
#     title = str(params) + '_t-SNE'
#     sc.pl.tsne(tsne_adata, color="tumor_groups", title=title)#, add_outline=True, size=50, palette="tab20")


sc.tl.tsne(tsne_adata, learning_rate=lr, n_pcs = 9, perplexity=perpl )
sc.pl.tsne(tsne_adata, color="tumor_groups", title='t-SNE', add_outline=True, size=50, palette="tab20")


In [ ]:
sc.tl.tsne(cadata, learning_rate=lr, n_pcs = 9, perplexity=perpl )

## Separation of Carcinoma and non Carcinoma cells 

In [ ]:
rest_rows.head()

In [ ]:
# Create a mapping dictionary from rest_rows
mapping_class = rest_rows.set_index('sample')['index'].to_dict()
mapping_type = rest_rows.set_index('sample')['index2'].to_dict()
mapping_spec = rest_rows.set_index('sample')['index3'].to_dict()

cadata.obs['class'] = cadata.obs_names.map(mapping_class)
cadata.obs['type'] = cadata.obs_names.map(mapping_type)
cadata.obs['spec'] = cadata.obs_names.map(mapping_spec)

In [ ]:
# Same for padata
mapping_class = bulk_rows.set_index('sample')['index'].to_dict()
mapping_type = bulk_rows.set_index('sample')['index2'].to_dict()
mapping_spec = bulk_rows.set_index('sample')['index3'].to_dict()

padata.obs['class'] = padata.obs_names.map(mapping_class)
padata.obs['type'] = padata.obs_names.map(mapping_type)
padata.obs['spec'] = padata.obs_names.map(mapping_spec)

In [ ]:
sc.pl.umap(cadata, color="class", title='Carcinoma/Non carcinoma cells/UMAP', add_outline=True, size = 50,palette = "tab20")

In [ ]:
sc.pl.tsne(cadata, color="class", title='Carcinoma/Non carcinoma cells/TSNE', add_outline=True, size = 50,palette = "tab20")

In [ ]:
sc.pl.pca(cadata, color="class", title='Carcinoma/Non carcinoma cells/PCA', add_outline=True, size = 50,palette = "tab20")

In [ ]:
from sklearn.cluster import AgglomerativeClustering

data = cadata.X
labels = (cadata.obs["class"] == "Tumor").astype(int)

# Apply hierarchical clustering with 2 clusters
model = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
model.fit(data)

predicted_labels = model.labels_

# Compare predicted labels with actual labels
accuracy = (predicted_labels == labels).mean()
print(f"Accuracy: {accuracy}")


In [ ]:
cadata.obs['Labels'] = predicted_labels.astype(str)

# Plot the gene expression heatmap with cluster annotations
sc.pl.heatmap(cadata, var_names=cadata.var_names, groupby='Labels', cmap='viridis',
             dendrogram=True, show=True, figsize=(12, 8))

In [ ]:
sc.pl.scatter(cadata,x='log1p_n_genes_by_counts', y='log1p_total_counts_mt', color='class', legend_fontsize=10, size = 100, show=True, title='Scatter plot of cancer & non cancer cells' )

In [ ]:
sc.pl.clustermap(cadata)